In [1]:
import pandas as pd
import jieba
import string

from sklearn.preprocessing import MultiLabelBinarizer

In [2]:
df = pd.read_csv('category_data\cat_train_v2.csv')
df = df[['name', 'product']]
'''已知問題資料: 9691, 法文, 日文'''

'已知問題資料: 9691, 法文, 日文'

In [3]:
df = df[df['name'] != '#NAME?']
df.reset_index(inplace=True)
df.drop(['index'], axis=1, inplace=True)
# df.set_index(['index'], inplace=True)

In [4]:
with open('stop.txt', 'r', encoding='utf-8') as f:
    stop = [i.strip() for i in f.readlines()]

In [5]:
df['name'] = df['name'].str.replace('\W', '', regex=True)

In [6]:
def remove_things(text):
    text = "".join([i for i in text if i not in stop])               # 移除停止詞
    text = "".join([i for i in text if i not in string.punctuation]) # 移除標點
    text = "".join([i for i in text if i not in string.whitespace])  # 移除空格
    return text

def strQ2B(ustring):
    """把字串全形轉半形"""
    ss = []
    for s in ustring:
        rstring = ""
        for uchar in s:
            inside_code = ord(uchar)
            if inside_code == 12288:  # 全形空格直接轉換
                inside_code = 32
            elif (inside_code >= 65281 and inside_code <= 65374):  # 全形字元（除空格）根據關係轉化
                inside_code -= 65248
            rstring += chr(inside_code)
        ss.append(rstring)
    return ''.join(ss)

In [7]:
df['name'] = df['name'].apply(lambda x: x.lower())
df['name'] = df['name'].apply(lambda x: remove_things(x))
df['name'] = df['name'].apply(lambda x: strQ2B(x))

In [8]:
df = df[df['name'] != '']
df.reset_index(inplace=True)
df.drop(['index'], axis=1, inplace=True)

In [9]:
mlb = MultiLabelBinarizer()
mlb_result = mlb.fit_transform([str(df.loc[i, 'product']).split(';') for i in range(len(df))])

# Add words to jieba dictionary

In [12]:
brand = pd.read_csv('brand_data\\brand_train.csv')

,name,occurred_at,iv_price,brand,units,unit_price,total_price,channel
0,元山熱水瓶YS-540AP,2021-02-07,1650,元山,1.0,1490.0,1188.0,738fbc9320a9106d8905bef71a02bb99
1,356612@姍拉娜治痘洗面乳150G,2021-02-02,1053,姍拉娜,1.0,85.0,85.0,d10c78673e1d88714ba645fadc3de0d1
2,台鹽海洋鹼性離子水 600ML,2021-02-04,609,台鹽,1.0,19.0,19.0,d7769307282f69448322f9306bdf9700
3,妮傲絲翠果酸深層保養乳液,2021-02-14,1142,NEO-TEC妮傲絲翠,1.0,1200.0,1200.0,7a0a2154662bc9038594f04c886c87d8
4,舒潔棉柔舒適迪士尼抽取式衛生紙 100抽16入,2021-02-01,198,舒潔,1.0,198.0,198.0,ab7febfe145bba3da1c0fb19507f1703
...,...,...,...,...,...,...,...,...
70041,惠而浦16公斤瓦斯型滾筒蒸氣(可堆疊)乾衣機8TWGD6622HW,2021-02-07,35291,惠而浦 Whirlpool,1.0,35291.0,35291.0,4a3d67cc0cd76bf56f18b40ed671bb05
70042,康乃馨成人紙尿褲L,2021-02-04,1041,康乃馨,1.0,194.0,194.0,9742158c78b35d77c6351ecbfb60897b
70043,日本芮芙茹零矽靈洗髮露460ml頭皮保養,2021-02-04,299,芮芙茹 Reveur,1.0,299.0,299.0,0f20279a949fd072f0e3858f97d0654a
70044,Crest長效鎖白牙膏-輕柔鑽白99g,2021-02-13,401,crest,1.0,99.0,99.0,6757b9193b3446acb4b97a05649b3c7e


In [25]:
test_list = list(set(brand.loc[:, 'brand'].values))

In [29]:
new_list = []
for i in test_list:
    for j in i.split():
        new_list.append(j)

In [31]:
new_list = [i.lower() for i in new_list]

In [33]:
final_list = []
for word in new_list:
    for letter in word:
        if letter in string.punctuation:
            word = word.replace(letter, '')
    final_list.append(word)

final_list

['國際牌',
 '加倍潔',
 '帕米爾',
 'parmir',
 '比依',
 'clio',
 '珂莉奧',
 'muji',
 '無印良品',
 '露遊',
 'treewalker',
 'dr',
 '藝群',
 'reebok',
 'heme',
 '喜蜜',
 'lemin立明',
 '星巴克',
 '紅薑黃先生',
 'drkonopka’s',
 'infinity',
 '無限肌緻',
 'sylph',
 '范倫鐵諾',
 'valentino',
 'rudy',
 '台畜',
 '惠普',
 'hp',
 'tresemme',
 '北海道函館37鮮乳',
 'msh',
 'weleda',
 '金寶',
 'under',
 'armour',
 'ogeu',
 '芙夏妮',
 '姍莎菈',
 '高大鮮乳',
 '理想',
 'perfect',
 '金美克能',
 'famiclean',
 '全家安',
 'sea',
 'to',
 'summit',
 '醬本缸',
 'key',
 'coffee',
 'teteo',
 '奇異',
 '比菲多乳之初',
 'pc',
 'park',
 'nailmatic',
 'lcdp',
 '風騰',
 '味榮',
 '海爾',
 'haier',
 '萃樂口',
 '妙奇',
 'moggy',
 '愛戀咖啡',
 '台菱',
 'paddy',
 '荷蘭公主',
 'royal',
 'unibrew',
 'bb',
 'love',
 'be',
 'quiet',
 '囍碗',
 'loveabowl',
 'new',
 'anna',
 'head',
 '海德',
 'nuxe',
 '黎可詩',
 '味全簡單點',
 '米徠',
 'golden',
 'switch',
 'minon',
 'aminomoist',
 '蜜濃',
 'future',
 'lab',
 '未來實驗室',
 '萊克蒂',
 '優力巾',
 'avengers',
 '哈嘍24h',
 '台東初鹿',
 'sanbisi',
 '安雅',
 'sparkeau',
 'oneroad',
 'romix',
 '佐藤',
 '伊德爾',
 'lotus',
 '樂天',


In [34]:
for i in final_list:
    jieba.add_word(i)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\dddru\AppData\Local\Temp\jieba.cache
Loading model cost 1.028 seconds.
Prefix dict has been built successfully.


In [35]:
for i in mlb.classes_:
    jieba.add_word(i)

In [36]:
cut_all_false = []
cut_all_true = []
HMM_false = []
HMM_true = []
paddle_false = []
paddle_true = []
all_false = []
all_true = []

for i in range(len(df)):
    cut_all_false.append(','.join(jieba.cut(df.name[i], cut_all=False, HMM=True, use_paddle=True)))
    cut_all_true.append(','.join(jieba.cut(df.name[i], cut_all=True, HMM=False, use_paddle=False)))
    HMM_false.append(','.join(jieba.cut(df.name[i], cut_all=True, HMM=False, use_paddle=True)))
    HMM_true.append(','.join(jieba.cut(df.name[i], cut_all=False, HMM=True, use_paddle=False)))
    paddle_false.append(','.join(jieba.cut(df.name[i], cut_all=True, HMM=True, use_paddle=False)))
    paddle_true.append(','.join(jieba.cut(df.name[i], cut_all=False, HMM=False, use_paddle=True)))
    all_false.append(','.join(jieba.cut(df.name[i], cut_all=False, HMM=False, use_paddle=False)))
    all_true.append(','.join(jieba.cut(df.name[i], cut_all=True, HMM=True, use_paddle=True)))

In [17]:
jb = pd.DataFrame({'cut_all_false':cut_all_false,
                    'cut_all_true': cut_all_true,
                    'HMM_false': HMM_false,
                    'HMM_true': HMM_true,
                    'paddle_false': paddle_false,
                    'paddle_true': paddle_true,
                    'all_false': all_false,
                    'all_true': all_true})
df = pd.concat([df, jb], axis=1)
df_final = pd.concat([df.drop(['product'], axis=1), pd.DataFrame(mlb_result, columns=list(mlb.classes_))], axis=1)
df_final.to_csv('preprocessed_jieba_cat_train.csv', index=False)